In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import scipy as sp
from IPython.display import Image

## Linearization and shooting methods

$ x''=1-e^x, x(0)=x(120)=0, x'(0)=x'(12)=A \geq 10 $

In [3]:
def f_calc_d2(x):
    return 1-np.exp(x)

def f_calc_d1(x):
    x, dx = x
    return np.array((dx, f_calc_d2(x)))

### Shooting algorithm

Choose $\alpha_0 = x(120)-x(0) = 0$ and solve Cauchy-problem with $x'(0) = \alpha_0 $. Find y(120) for this problem. Correct $\alpha_0$ in corresponding direction if you're not satisfied.

In [14]:
def runge_kutt_iter(y0, h, f_calc): 
    ## one step of modified Euler's method with correction
    y1 = y0 + h * f_calc(y0)
    y1 = y0 + (h/2 * (f_calc(y0)+f_calc(y1)))
    return y1

def calc_bdf(tmin, tmax, x0, n, f_calc):
    h = (tmax - tmin)/n
    t = np.linspace(tmin, tmax, n+1, endpoint=True)
    dim = len(x0)
    y = np.zeros((n+1, dim))
    y[0] = x0
    y[1] = runge_kutt_iter(y[0], h, f_calc)
    for i in range(2, n+1):
        to_solve = lambda yi: -yi + (4/3) * y[i-1] - (1/3) * y[i-2] + (2/3) * h * f_calc(yi)
        y[i] = sp.optimize.fsolve(to_solve, y[i-1])
    return t, y

#### General example of solutions

In [92]:
for alpha in [10, 40, 60]:
    px.line(y=calc_bdf(0, 120, np.array((0, alpha)), 1000, f_calc_d1)[1][:, 0], title=str(alpha)).show()

In [56]:
def find_last_root_pos_df(arr, atol=1):
    # finds the most right root with positive derivative
    roots = np.where(np.isclose(arr, 0, atol=atol))[0]
    if len(roots)==1:
        return 0
    for ind in roots[::-1]:
        if ind==len(arr)-1:
            deriv = arr[ind]-arr[ind-1]
        else:
            deriv = (arr[ind+1]-arr[ind-1])/2
        if deriv>0:
            return ind
    return 0

In [79]:
def find_all_a(alpha0=60):
    good_a = []
    roots = []
    for alpha in range(alpha0, 9, -1):
        res = calc_bdf(0, 120, np.array((0, alpha)), 1000, f_calc_d1)[1][:, 0]
        # print(f'{a=}, {find_last_root_pos_df(res)}')
        roots.append(find_last_root_pos_df(res))
        if find_last_root_pos_df(res)>950:
            good_a.append(alpha)
        # px.line(y=res, title=str(alpha)).show()
    return good_a

#### First approximations of solutions

In [81]:
ans = find_all_a()
ans

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning:

The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.



[38, 37, 26, 25, 18, 14, 11]

In [93]:
for alpha in ans:
    px.line(y=calc_bdf(0, 120, np.array((0, alpha)), 1000, f_calc_d1)[1][:, 0], title=str(alpha)).show()

#### Correction

In [102]:
def bsearch_a(la, ra):
    lpos = find_last_root_pos_df(
        calc_bdf(0, 120, np.array((0, la)), 1000, f_calc_d1)[1][:, 0]
    )
    if lpos>997:
        return la
    rpos = find_last_root_pos_df(
        calc_bdf(0, 120, np.array((0, ra)), 1000, f_calc_d1)[1][:, 0]
    )
    if rpos>997:
        return ra
    if rpos > lpos: 
        print(ra, la, rpos, lpos)
        raise Exception('your boundaries are on the same side')
    ha = (ra+la)/2
    hpos = find_last_root_pos_df(
        calc_bdf(0, 120, np.array((0, ha)), 1000, f_calc_d1)[1][:, 0]
    )
    print(la, ra, lpos, rpos, ha, hpos)
    if hpos < 950: # too big period~a
        return bsearch_a(la, ha)
    return bsearch_a(ha, ra) # too small period~a

In [103]:
alphas = [38, 26, 18, 14, 11] # some are dropped because dupliactes
corrected = [] 
for alpha in alphas:
    corrected.append(bsearch_a(alpha, alpha+2))
print(corrected)

38 40 988 0 39.0 0
38 39.0 988 0 38.5 553
38 38.5 988 553 38.25 995
38.25 38.5 995 553 38.375 551
38.25 38.375 995 551 38.3125 996
38.3125 38.375 996 551 38.34375 997
38.34375 38.375 997 551 38.359375 998
26 28 997 750 27.0 728
26 27.0 997 728 26.5 718
26 26.5 997 718 26.25 712
26 26.25 997 712 26.125 1000
18 20 975 822 19.0 792
18 19.0 975 792 18.5 994
18.5 19.0 994 792 18.75 784
18.5 18.75 994 784 18.625 998
14 16 994 898 15.0 858
14 15.0 994 858 14.5 838
14 14.5 994 838 14.25 828
14 14.25 994 828 14.125 1000
11 13 990 945 12.0 894
11 12.0 990 894 11.5 868
11 11.5 990 868 11.25 855
11 11.25 990 855 11.125 998
[38.359375, 26.125, 18.625, 14.125, 11.125]


In [104]:
for alpha in corrected:
    px.line(y=calc_bdf(0, 120, np.array((0, alpha)), 1000, f_calc_d1)[1][:, 0], title=str(alpha)).show()